In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLOv8 model
model = YOLO('../models/yolov8n.pt')  # Use 'yolov8s.pt' etc. as needed

cap = cv2.VideoCapture(1)

ret, frame1 = cap.read()
ret2, frame2 = cap.read()

if not ret or not ret2:
    print("Failed to grab initial frames.")
    cap.release()
    exit()

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False
    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue
        motion_detected = True
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if motion_detected:
        # Run YOLOv8 on the frame
        results = model(frame1, verbose=False)
        annotated_frame = results[0].plot()
        cv2.imshow("Motion + YOLO", annotated_frame)
    else:
        cv2.imshow("Motion + YOLO", frame1)

    frame1 = frame2
    ret, frame2 = cap.read()
    if not ret:
        break

    if cv2.waitKey(10) == 27:
        break

cap.release()
cv2.destroyAllWindows()